In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [107]:
#Put unlimited max columns when displaying dataframes
pd.set_option('display.max_columns', None)

In [108]:
df2023_2024 = pd.read_excel("DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx")
df2022_2023 = pd.read_excel("DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx", sheet_name=None)
df2020_2021 = pd.read_excel("DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx", sheet_name=None)


# Parte 1
## A) Dimensión de la base de datos

In [109]:
#Obtener shape de cada dataframe
print("Shape de df2023_2024: ", df2023_2024.shape)
for key in df2022_2023:
    print("Shape de df2022_2023: ", key , df2022_2023[key].shape)
for key in df2020_2021:
    print("Shape de df2020_2021: ", key , df2020_2021[key].shape)


Shape de df2023_2024:  (13872, 240)
Shape de df2022_2023:  SURESTE (14255, 16)
Shape de df2022_2023:  NORESTE (14255, 16)
Shape de df2022_2023:  CENTRO (14255, 16)
Shape de df2022_2023:  NOROESTE (14255, 16)
Shape de df2022_2023:  SUROESTE (14255, 16)
Shape de df2022_2023:  NOROESTE2 (14255, 16)
Shape de df2022_2023:  NORTE (14255, 16)
Shape de df2022_2023:  SUROESTE2 (14255, 16)
Shape de df2022_2023:  SURESTE2 (14255, 16)
Shape de df2022_2023:  SURESTE3 (14255, 16)
Shape de df2022_2023:  SUR (14254, 16)
Shape de df2022_2023:  NORTE2 (14255, 16)
Shape de df2022_2023:  NORESTE2 (14255, 16)
Shape de df2022_2023:  NORESTE3 (14254, 16)
Shape de df2022_2023:  NOROESTE3 (6237, 16)
Shape de df2022_2023:  CATÁLOGO (15, 3)
Shape de df2020_2021:  SURESTE (17538, 16)
Shape de df2020_2021:  NORESTE (17535, 16)
Shape de df2020_2021:  CENTRO (17536, 16)
Shape de df2020_2021:  NOROESTE (17537, 16)
Shape de df2020_2021:  SUROESTE (17537, 16)
Shape de df2020_2021:  NOROESTE2 (17535, 16)
Shape de df2020

## B) Describe las variables, incluyendo nombre, descripción, tipo, valores posibles y nulos.

## C) Verifica la calidad de los datos: valores faltantes, valores de los datos, valores espurios o erróneos

# Parte 2
## A) Selecciona el conjunto de datos a utilizar
Decide qué conjunto de datos se utilizará. Explica por qué se incluyeron o excluyeron ciertos datos.
Identifica las columnas objetivo


Bases de datos seleccionadas: Datos historicos 2023-2024, Datos historicos 2022-2023, Datos historicos 2020-2021 y datos históricos 2021.
Estas bases se seleccionaron pues son las más recientes, completas y presentan un formato similar para su análisis. Al ser series de tiempo se pueden juntar entre ellas para hacer un análisis más completo.
Las columnas objetivo seria tener cada columna con el formato "Estación Contaminante (medida)", indexados por su fecha, de manera que se puedan almacenar los datos como una serie de tiempo durante los 4 años.

# Hacer columnas similares para cada base de datos

Preparación de 2023-2024

In [110]:
df2023_2024.loc[0,:] = df2023_2024.loc[0,:].fillna("A")
df2023_2024.columns = df2023_2024.columns.str.split('.').str[0]
df2023_2024 = df2023_2024.replace("WDV", "WDR")
df2023_2024.columns = (df2023_2024.columns + " " + df2023_2024.loc[0] + " (" + df2023_2024.loc[1] +")").str.upper()
df2023_2024.drop([0,1], inplace=True)
df2023_2024.rename(columns={"UNNAMED: 0 A (DATE)": "FECHA"}, inplace=True)
df2023_2024["FECHA"] = pd.to_datetime(df2023_2024["FECHA"])
#Rename the columns that have "sureste2" to "sureste 2"
df2023_2024.columns = df2023_2024.columns.str.replace("SURESTE2", "SURESTE 2")
df2023_2024.columns = df2023_2024.columns.str.replace("NORESTE2", "NORESTE 2")
df2023_2024.columns = df2023_2024.columns.str.replace("SUROESTE2", "SUROESTE 2")

C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\1579210948.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2023_2024.loc[0,:] = df2023_2024.loc[0,:].fillna("A")
C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\1579210948.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2023_2024.loc[0,:] = df2023_2024.loc[0,:].fillna("A")
C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\1579210948.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A' has dtype incompatible with float64, please explicitly cas

In [111]:
df2023_2024

,FECHA,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NaN,NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),NaN,CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NaN,NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),NaN,SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NaN,NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NaN,NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),NaN,NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),NaN,SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),NaN,SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),NaN,SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),NaN,SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NaN,NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NaN,NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PM2.5 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE

Preparación de 2022-2023

In [112]:
# Create a new dictionary to store modified keys
modified_keys = {}

for key in df2022_2023.keys():
    # Check if the last character is a digit
    if key[-1].isdigit():
        # Modify the key as the original name minus the digit + "digit"
        new_key = key[:-1] + " " + key[-1]
    else:
        # Keep the original key
        new_key = key
    # Add to the modified keys dictionary
    modified_keys[key] = new_key

# Change the keys of df 2022_2023 with the modified keys
df2022_2023 = {modified_keys[key]: df for key, df in df2022_2023.items()}
#Add the name of the sheet 2022_2023 and the unit associated with the column to the column names 
units = {"CO": "(PPM)", "CO2": "(PPM)", "NO": "(PPB)", "NO2": "(PPB)", "NOX": "(PPB)", "O3": "(PPB)", "PM10": "(UG/M3)", "PM2.5": "(UG/M3)", "SO2": "(PPB)", "PRS": "(MMHG)", "RH": "(%)", "TOUT": "(DEGC)", "WSP": "(M/S)", "RAINF": "(MM/HR)", "WD": "(DEG)", "WSR": "(KMPH)", "WDR": "(DEG)", "SR": "(KW/M2)"}
df2022_2023.keys()
for key in df2022_2023.keys():
    # Add the unit to the column names 
    for column in df2022_2023[key].columns:
        if column in units:
            df2022_2023[key].rename(columns={column: key+ " " + column + " " + units[column]}, inplace=True)
        

df2022_2023["SUROESTE"]
#Join all the dataframes in the dictionary by the date column
from functools import reduce

# Example dictionary of DataFrames (df2022_2023)
# Assuming df2022_2023 contains all your DataFrames

# Filter out 'CATÁLOGO' and store the DataFrames to be merged
dfs_to_merge = [df2022_2023[key] for key in df2022_2023 if key != 'CATÁLOGO']

# Merge all DataFrames on the 'date' column
df2022_2023_merged = reduce(lambda left, right: pd.merge(left, right, on='date',how="outer"), dfs_to_merge)

# Output the merged DataFrame
df2022_2023_merged.rename(columns={"date": "FECHA"}, inplace=True)
df2022_2023_merged["FECHA"] = pd.to_datetime(df2022_2023_merged["FECHA"])
df2022_2023_merged

,FECHA,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PM2.5 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),N

In [113]:
df2022_2023["NORESTE 3"]

,date,NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PM2.5 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NORESTE 3 WSR (KMPH),NORESTE 3 WDR (DEG)
0,2022-01-01 00:00:00,0.544,8.0,9.7,17.0,19.9,54.0,NaN,723.0,0.0,75.0,4.4,0.000,19.6,1.1,286.0
1,2022-01-01 01:00:00,0.440,5.4,7.1,12.0,21.5,14.0,NaN,729.0,0.0,82.0,3.4,0.000,5.1,8.3,126.0
2,2022-01-01 02:00:00,0.711,9.5,15.7,24.6,5.2,74.0,NaN,722.0,0.0,85.0,4.4,0.000,16.5,1.0,282.0
3,2022-01-01 03:00:00,0.842,10.6,16.8,26.7,4.1,67.0,NaN,722.0,0.0,87.0,4.4,0.000,15.8,1.0,282.0
4,2022-01-01 04:00:00,0.914,10.0,17.5,26.8,4.4,73.0,NaN,722.0,0.0,87.0,4.6,0.000,15.9,1.0,289.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14249,2023-08-17 19:00:00,NaN,NaN,NaN,NaN,NaN,35.0,NaN,726.0,0.0,18.0,NaN,0.106,38.1,14.3,58.0
14250,2023-08-17 20:00:00,NaN,NaN,NaN,NaN,NaN,33.0,NaN,726.0,0.0,20.0,NaN,0.002,36.2,10.6,53.0
14251,2023-08-17 21:00:00,NaN,NaN,NaN,NaN,NaN,43.0,NaN,727.0,0.0,23.0,NaN,0.000,34.8,8.4,48.0
14252,2023-08-17 22:00:00,NaN,NaN,NaN,NaN,NaN,52.0,NaN,728.0,0.0,34.0,NaN,0.000,33.4,13.9,61.0


Preparación de 2021-2022

In [114]:
# Create a new dictionary to store modified keys
modified_keys = {}

for key in df2020_2021.keys():
    # Check if the last character is a digit
    if key[-1].isdigit():
        # Modify the key as the original name minus the digit + "digit"
        new_key = key[:-1] + " " + key[-1]
    else:
        # Keep the original key
        new_key = key
    # Add to the modified keys dictionary
    modified_keys[key] = new_key

# Change the keys of df 2020_2021 with the modified keys
df2020_2021 = {modified_keys[key]: df for key, df in df2020_2021.items()}

for key in df2020_2021.keys():
    # Add the unit to the column names 
    for column in df2020_2021[key].columns:
        if column in units:
            df2020_2021[key].rename(columns={column: key + " " + column + " " + units[column]}, inplace=True)
#Merge all the keys of 2020_2021 dataframes, minus the catalog
#Join all the dataframes in the dictionary by the date column

# Example dictionary of DataFrames (df2022_2023)
# Assuming df2022_2023 contains all your DataFrames

# Filter out 'CATÁLOGO' and "NOROESTE 3" and store the DataFrames to be merged
dfs_to_merge = [df2020_2021[key] for key in df2020_2021 if key not in ['CATÁLOGO', 'NOROESTE 3']]

# Merge all DataFrames on the 'date' column
df2020_2021_merged = reduce(lambda left, right: pd.merge(left, right, on='date', how="outer"), dfs_to_merge)

# Output the merged DataFrame
df2020_2021_merged.rename(columns={"date": "FECHA"}, inplace=True)
df2020_2021_merged["FECHA"] = pd.to_datetime(df2020_2021_merged["FECHA"])
df2020_2021_merged

,FECHA,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PM2.5 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),N

In [115]:
#Dimensiones de los datasets
print("Dimensiones de df2023_2024: ", df2023_2024.shape)
print("Dimensiones de df2022_2023_merged: ", df2022_2023_merged.shape)
print("Dimensiones de df2020_2021_merged: ", df2020_2021_merged.shape)


Dimensiones de df2023_2024:  (13870, 240)
Dimensiones de df2022_2023_merged:  (14255, 226)
Dimensiones de df2020_2021_merged:  (17544, 211)


In [116]:
#Se eliminan las columnas que no tienen al menos el 2% de los datos
df2023_2024 = df2023_2024.dropna(thresh=0.02*len(df2023_2024), axis=1)
df2022_2023_merged = df2022_2023_merged.dropna(thresh=0.02*len(df2022_2023_merged), axis=1)
df2020_2021_merged = df2020_2021_merged.dropna(thresh=0.02*len(df2020_2021_merged), axis=1)


In [117]:
#Nuevas dimensiones de los datasets
print("Dimensiones de df2023_2024: ", df2023_2024.shape)
print("Dimensiones de df2022_2023_merged: ", df2022_2023_merged.shape)
print("Dimensiones de df2020_2021_merged: ", df2020_2021_merged.shape)


Dimensiones de df2023_2024:  (13870, 225)
Dimensiones de df2022_2023_merged:  (14255, 224)
Dimensiones de df2020_2021_merged:  (17544, 210)


In [118]:
#Hacer las fechas datetime
df2023_2024["FECHA"] = pd.to_datetime(df2023_2024["FECHA"])
df2022_2023_merged["FECHA"] = pd.to_datetime(df2022_2023_merged["FECHA"])
df2020_2021_merged["FECHA"] = pd.to_datetime(df2020_2021_merged["FECHA"])

C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\41385782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2023_2024["FECHA"] = pd.to_datetime(df2023_2024["FECHA"])
C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\41385782.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2022_2023_merged["FECHA"] = pd.to_datetime(df2022_2023_merged["FECHA"])
C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\41385782.py:4: SettingWithCopyWarning: 
A value is tryi

Unir las bases de datos por fecha

In [119]:

df2022_merged = df2022_2023_merged.query("FECHA < '2023-01-01'")

In [120]:
df2022_merged

,FECHA,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NOR

In [121]:
df2023_2024

,FECHA,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NOR

In [122]:
dfs_to_merge = [df2023_2024, df2022_merged, df2020_2021_merged]

# Merge all DataFrames on the 'date' column
df_time_serie = pd.concat([dfs_to_merge[0], dfs_to_merge[1], dfs_to_merge[2]], axis=0)

In [123]:
df_time_serie.set_index("FECHA", inplace=True)

In [124]:
#Ordenar por fecha
df_time_serie.sort_index(inplace=True)

In [125]:
df_time_serie

,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NORESTE 3

In [126]:
# Fill data with the mobile average of the last 6 hours
df_time_serie.replace(-9999, np.nan, inplace=True)
df_time_serie = df_time_serie.fillna(df_time_serie.rolling(6, min_periods=1).mean())


C:\Users\progra.DESKTOP-GV4Q93K\AppData\Local\Temp\ipykernel_10980\3935799380.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_time_serie.replace(-9999, np.nan, inplace=True)


In [127]:
df_time_serie_2022 =df_time_serie["2022-01-01":]
df_time_serie_2022

,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NORESTE 3

In [128]:
df_time_serie_2022_nonans = df_time_serie_2022.fillna(df_time_serie_2022.rolling(24, min_periods=1).mean())

In [129]:
df_time_serie_2022_nonans

,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NORESTE 3

In [130]:
df_time_serie_2022_nonans_nor3 = df_time_serie_2022_nonans[df_time_serie_2022_nonans.columns[~df_time_serie_2022_nonans.columns.str.contains('NOROESTE 3.*')]]

In [131]:
df_time_serie_2022_nonans_nor3

,SURESTE CO (PPM),SURESTE NO (PPB),SURESTE NO2 (PPB),SURESTE NOX (PPB),SURESTE O3 (PPB),SURESTE PM10 (UG/M3),SURESTE PM2.5 (UG/M3),SURESTE PRS (MMHG),SURESTE RAINF (MM/HR),SURESTE RH (%),SURESTE SO2 (PPB),SURESTE SR (KW/M2),SURESTE TOUT (DEGC),SURESTE WSR (KMPH),SURESTE WDR (DEG),NORESTE CO (PPM),NORESTE NO (PPB),NORESTE NO2 (PPB),NORESTE NOX (PPB),NORESTE O3 (PPB),NORESTE PM10 (UG/M3),NORESTE PM2.5 (UG/M3),NORESTE PRS (MMHG),NORESTE RAINF (MM/HR),NORESTE RH (%),NORESTE SO2 (PPB),NORESTE SR (KW/M2),NORESTE TOUT (DEGC),NORESTE WSR (KMPH),NORESTE WDR (DEG),CENTRO CO (PPM),CENTRO NO (PPB),CENTRO NO2 (PPB),CENTRO NOX (PPB),CENTRO O3 (PPB),CENTRO PM10 (UG/M3),CENTRO PM2.5 (UG/M3),CENTRO PRS (MMHG),CENTRO RAINF (MM/HR),CENTRO RH (%),CENTRO SO2 (PPB),CENTRO SR (KW/M2),CENTRO TOUT (DEGC),CENTRO WSR (KMPH),CENTRO WDR (DEG),NOROESTE CO (PPM),NOROESTE NO (PPB),NOROESTE NO2 (PPB),NOROESTE NOX (PPB),NOROESTE O3 (PPB),NOROESTE PM10 (UG/M3),NOROESTE PM2.5 (UG/M3),NOROESTE PRS (MMHG),NOROESTE RAINF (MM/HR),NOROESTE RH (%),NOROESTE SO2 (PPB),NOROESTE SR (KW/M2),NOROESTE TOUT (DEGC),NOROESTE WSR (KMPH),NOROESTE WDR (DEG),SUROESTE CO (PPM),SUROESTE NO (PPB),SUROESTE NO2 (PPB),SUROESTE NOX (PPB),SUROESTE O3 (PPB),SUROESTE PM10 (UG/M3),SUROESTE PM2.5 (UG/M3),SUROESTE PRS (MMHG),SUROESTE RAINF (MM/HR),SUROESTE RH (%),SUROESTE SO2 (PPB),SUROESTE SR (KW/M2),SUROESTE TOUT (DEGC),SUROESTE WSR (KMPH),SUROESTE WDR (DEG),NOROESTE 2 CO (PPM),NOROESTE 2 NO (PPB),NOROESTE 2 NO2 (PPB),NOROESTE 2 NOX (PPB),NOROESTE 2 O3 (PPB),NOROESTE 2 PM10 (UG/M3),NOROESTE 2 PM2.5 (UG/M3),NOROESTE 2 PRS (MMHG),NOROESTE 2 RAINF (MM/HR),NOROESTE 2 RH (%),NOROESTE 2 SO2 (PPB),NOROESTE 2 SR (KW/M2),NOROESTE 2 TOUT (DEGC),NOROESTE 2 WSR (KMPH),NOROESTE 2 WDR (DEG),NORTE CO (PPM),NORTE NO (PPB),NORTE NO2 (PPB),NORTE NOX (PPB),NORTE O3 (PPB),NORTE PM10 (UG/M3),NORTE PM2.5 (UG/M3),NORTE PRS (MMHG),NORTE RAINF (MM/HR),NORTE RH (%),NORTE SO2 (PPB),NORTE SR (KW/M2),NORTE TOUT (DEGC),NORTE WSR (KMPH),NORTE WDR (DEG),NORESTE 2 CO (PPM),NORESTE 2 NO (PPB),NORESTE 2 NO2 (PPB),NORESTE 2 NOX (PPB),NORESTE 2 O3 (PPB),NORESTE 2 PM10 (UG/M3),NORESTE 2 PM2.5 (UG/M3),NORESTE 2 PRS (MMHG),NORESTE 2 RAINF (MM/HR),NORESTE 2 RH (%),NORESTE 2 SO2 (PPB),NORESTE 2 SR (KW/M2),NORESTE 2 TOUT (DEGC),NORESTE 2 WSR (KMPH),NORESTE 2 WDR (DEG),SURESTE 2 CO (PPM),SURESTE 2 NO (PPB),SURESTE 2 NO2 (PPB),SURESTE 2 NOX (PPB),SURESTE 2 O3 (PPB),SURESTE 2 PM10 (UG/M3),SURESTE 2 PM2.5 (UG/M3),SURESTE 2 PRS (MMHG),SURESTE 2 RAINF (MM/HR),SURESTE 2 RH (%),SURESTE 2 SO2 (PPB),SURESTE 2 SR (KW/M2),SURESTE 2 TOUT (DEGC),SURESTE 2 WSR (KMPH),SURESTE 2 WDR (DEG),SUROESTE 2 CO (PPM),SUROESTE 2 NO (PPB),SUROESTE 2 NO2 (PPB),SUROESTE 2 NOX (PPB),SUROESTE 2 O3 (PPB),SUROESTE 2 PM10 (UG/M3),SUROESTE 2 PM2.5 (UG/M3),SUROESTE 2 PRS (MMHG),SUROESTE 2 RAINF (MM/HR),SUROESTE 2 RH (%),SUROESTE 2 SO2 (PPB),SUROESTE 2 SR (KW/M2),SUROESTE 2 TOUT (DEGC),SUROESTE 2 WSR (KMPH),SUROESTE 2 WDR (DEG),SURESTE 3 CO (PPM),SURESTE 3 NO (PPB),SURESTE 3 NO2 (PPB),SURESTE 3 NOX (PPB),SURESTE 3 O3 (PPB),SURESTE 3 PM10 (UG/M3),SURESTE 3 PM2.5 (UG/M3),SURESTE 3 PRS (MMHG),SURESTE 3 RAINF (MM/HR),SURESTE 3 RH (%),SURESTE 3 SO2 (PPB),SURESTE 3 SR (KW/M2),SURESTE 3 TOUT (DEGC),SURESTE 3 WSR (KMPH),SURESTE 3 WDR (DEG),SUR CO (PPM),SUR NO (PPB),SUR NO2 (PPB),SUR NOX (PPB),SUR O3 (PPB),SUR PM10 (UG/M3),SUR PM2.5 (UG/M3),SUR PRS (MMHG),SUR RAINF (MM/HR),SUR RH (%),SUR SO2 (PPB),SUR SR (KW/M2),SUR TOUT (DEGC),SUR WSR (KMPH),SUR WDR (DEG),NORTE 2 CO (PPM),NORTE 2 NO (PPB),NORTE 2 NO2 (PPB),NORTE 2 NOX (PPB),NORTE 2 O3 (PPB),NORTE 2 PM10 (UG/M3),NORTE 2 PM2.5 (UG/M3),NORTE 2 PRS (MMHG),NORTE 2 RAINF (MM/HR),NORTE 2 RH (%),NORTE 2 SO2 (PPB),NORTE 2 SR (KW/M2),NORTE 2 TOUT (DEGC),NORTE 2 WSR (KMPH),NORTE 2 WDR (DEG),NORESTE 3 CO (PPM),NORESTE 3 NO (PPB),NORESTE 3 NO2 (PPB),NORESTE 3 NOX (PPB),NORESTE 3 O3 (PPB),NORESTE 3 PM10 (UG/M3),NORESTE 3 PRS (MMHG),NORESTE 3 RAINF (MM/HR),NORESTE 3 RH (%),NORESTE 3 SO2 (PPB),NORESTE 3 SR (KW/M2),NORESTE 3 TOUT (DEGC),NORESTE 3